### CV Assignment 2

1. **Mohamed Abdelaal El Sayed El Zarei 34-14613 T-13**
2. **Omar Ashraf Hussien Sallam 34-8761 T09**

In [48]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [49]:
def load_images():
    for i in range(1, 9):
        images.append(cv2.imread('./imagesA2/' + str(i) + '.jpg', 0))

#### Noise Detection & Removal

In [50]:
NOISE_THRESHOLD = 3000
def calc_and_fix_noise(kernel, img):
    width, height = img.shape
    filtered = np.array(cv2.blur(img,(kernel, kernel)), dtype=np.int32)
    variance = np.array((img - filtered), dtype=np.int32) ** 2
    noisy_pixels = variance[variance > NOISE_THRESHOLD].shape[0]
    percentage = (noisy_pixels / (width * height)) * 100
    print('- Noisy Pixels Percentage:', percentage,)
    if percentage > 8:
        print('===> Fixing noise ...')
        return cv2.medianBlur(img, kernel)
    return img

#### Blur Detection & Removal

In [51]:
def apply_edge_filter(kernel, img):
    Gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=kernel)
    Gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=kernel)
    G = abs(Gx) + abs(Gy)
    return G

In [94]:
def detect_and_fix_blur(img, k = 0.2, kernel = 3):
    width, height = img.shape
    G = apply_edge_filter(kernel, img)
    percentage = ((G[G < 10].shape[0]) / (width * height)) * 100
    laplacian_var = cv2.Laplacian(img, cv2.CV_64F).var()
    # detect using percentage of pixels or laplacian_var
    print('- Blurry Pixels Percentage:', percentage, '- Laplacian Variance:', laplacian_var)
    if percentage > 20 or laplacian_var < 100:
        print('===> Fixing blur ...')
        edges_sobel = apply_edge_filter(1, img)
#         smothed = cv2.blur(img,(3, 3))
#         edges = img - smothed
#         return img + (k * edges)
        return img + (k * edges_sobel)
    else:
        return img
        

#### Collapse Detection & Removal

In [95]:
def detect_and_fix_collapse(img):
    hist, bins = np.histogram(img, bins=range(256))
    width, height = img.shape
    used = np.nonzero(np.apply_along_axis(lambda x: x > 10, 0, hist))[0]
    c, d = used[0], used[used.shape[0] - 1] + 1
    a, b = 0, 255
    new_img = np.zeros(img.shape)
    print('- Percentage of color range:', (used.shape[0]/255) * 100)
    if used.shape[0] < 255:
        new_img = ((img - c) * ((b - a) / (d - c))) + a
        print('===> Fixing collapse ...')
        return new_img
    return img

### Main Call

In [96]:
def fix_image(img, name):
    im = calc_and_fix_noise(5, img)
    im = detect_and_fix_blur(im)
    im = detect_and_fix_collapse(im)
    cv2.imwrite(name + '.jpg', im)

In [98]:
images = []
load_images()
for i in range(len(images)):
    print('Image ' + str(i + 1))
    print('=' * 40)
    fix_image(images[i], './New_images/' + str(i + 1))

Image 1
- Noisy Pixels Percentage: 3.559375
- Blurry Pixels Percentage: 3.1681423611111112 - Laplacian Variance: 5148.6269343899985
- Percentage of color range: 100.0
Image 2
- Noisy Pixels Percentage: 12.710373263888888
===> Fixing noise ...
- Blurry Pixels Percentage: 15.59318576388889 - Laplacian Variance: 255.1190963275343
- Percentage of color range: 100.0
Image 3
- Noisy Pixels Percentage: 0.02634548611111111
- Blurry Pixels Percentage: 22.8109375 - Laplacian Variance: 51.73036396989533
===> Fixing blur ...
- Percentage of color range: 100.0
Image 4
- Noisy Pixels Percentage: 0.009244791666666667
- Blurry Pixels Percentage: 12.896831597222222 - Laplacian Variance: 827.7480235714987
- Percentage of color range: 42.35294117647059
===> Fixing collapse ...
Image 5
- Noisy Pixels Percentage: 8.978038194444444
===> Fixing noise ...
- Blurry Pixels Percentage: 26.711067708333335 - Laplacian Variance: 19.28725430380739
===> Fixing blur ...
- Percentage of color range: 100.0
Image 6
- Noi